In [102]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from sklearn import preprocessing
import pickle
from config import dropcol

In [103]:
load_path = 'intermediate_data/spec_20180808.pickle'

with open(load_path,'rb') as f:
    spec=pickle.load(f)

In [104]:
spec = pd.DataFrame(spec)

In [105]:
drop_col = [
            # 'price',
            'お名前(必須)',
            'お問合せカテゴリ(必須)',
            'その他費用',
            # 'バルコニー等面積',
            'メールアドレス※携帯アドレス不可',
            # '不動産会社詳細',
            # '交通',
            # '保証金・権利金',
            # '保険',
            '備考1',
            '備考2',
            '償却',
            # '入居時期',
            # '取引態様',
            # '向き',
            # '契約期間',
            '定期借家権契約',
            # '専有面積',
            '情報登録日',
            # '所在地',
            '敷引または解約引',
            # '敷金・礼金',
            # '更新料',
            # '構造',
            '次回更新予定日',
            '物件HP',
            # '物件種目',
            # '特記',
            # '現況',
            # '管理費等',
            # '築年月',
            '総戸数',
            '設備',
            # '詳細情報',
            # '間取り',
            '間取り内訳',
            # '階数',
            '雑費',
            '電話番号(必須)※携帯、IP可',
            '駐車場'    
            ]

In [106]:
def pre_price(df):
    df=df[~df['price'].isnull()]
    df=df.reset_index(drop=True)
    df['price']=df['price'].astype('str')
    df['price']=df['price'].str.replace(',','')
    price=[]
    for i in df['price']:
        if i.find('万') != -1:
            man,sen = i.split('万')
            if sen == '':
                sen = 0
            price.append(int(man)*10000+int(sen))
        else:
            price.append(int(i))
    df['price']=price
    return df

In [107]:
def pre_bid(df):
    df=df[~df['詳細情報'].isnull()]
    df=df.reset_index(drop=True)
    bid=[]
    for i,v in enumerate(df['詳細情報']):
        a,b=v.split('／')
        bid.append(a[len('物件番号:'):])
    df['詳細情報']=pd.DataFrame(bid)
    return df

In [108]:
def pre_balcony(df):
    col = 'バルコニー等面積'
    df[col] = df[col].fillna(0)
    df[col] = df[col].str.replace('㎡','')
    df[col] = df[col].str.replace('-','0')
    df[col] = df[col].astype(float)
    return df

In [109]:
def pre_transportation(df):
    station=[]
    method=[]
    time=[]
    for i,s in enumerate(df['交通']):
        f1=s.find('「')
        r1=s.find('」')
        station.append(s[f1+1:r1])
        rest=s[r1:]
        r2=rest.find('分')
        s1=s[r1:r1+r2]
        if s1.find('徒歩')!=-1:
            method.append('徒歩')
            try:
                time.append(int(s1[s1.find('徒歩')+2:]))
            except:
                time.append(np.nan)
        elif (s1.find('バス')!=-1)&(s1.find('バスにて')==-1):
            method.append('バス')
            time.append(int(s1[s1.find('バス')+2:]))
        elif s1.find('バスにて')!=-1:
            method.append('バス')
            time.append(int(s1[s1.find('バスにて')+4:]))
        else:
            method.append(np.nan)
            time.append(np.nan)

    df=df.drop('交通',axis=1)
    df['最寄り駅']=pd.DataFrame(station)
    df['駅からの手段']=pd.DataFrame(method)
    df['駅からの時間']=pd.DataFrame(time)

    df=df.dropna()
    df=df.reset_index(drop=True)

    return df

In [110]:
def pre_deposit(df):
    col = '保証金・権利金'
    df[col] = df[col].str.replace('保証金なし・-','0')
    df[col] = df[col].str.replace('-・-','0')
    df[col] = df[col].str.replace('円・-','')
    df[col] = df[col].str.replace(',','')
    price = []
    for i,v in enumerate(df[col]):
        if v.find('億') != -1:
            oku,man = v.split('億')
            if man == '':
                man = 0
            else:
                man = man[:-1]
            price.append(int(oku)*10000+int(man))
        elif v.find('ヶ月') != -1:
            p = df.loc[i,'price']
            m = float(v[:v.find('ヶ月')])
            price.append(p * m)
        elif v.find('万') != -1:
            man,sen = v.split('万')
            if sen == '':
                sen = 0
            price.append(int(man) + int(sen) / 10000)
        else:
            price.append(v)
    df[col] = price
    return df

In [111]:
def pre_insurance(df):
    col = '保険'
    ins = []
    for i,v in enumerate(df[col]):
        if v.find('有') != -1:
            ins.append('有')
        else:
            ins.append('無')
    df[col] = ins
    return df

In [112]:
def pre_timing(df):
    col = '入居時期'
    timing = []
    for i,v in enumerate(df[col]):
        if (v.find('相談') == -1) & (v.find('即時') == -1) :
            timing.append('その他')
        else:
            timing.append(v)
    df[col] = timing
    return df

In [113]:
def pre_term(df):
    col = '契約期間'
    term = []
    df[col] = df[col].str.replace('期間:','')
    for i,v in enumerate(df[col]):
        if (v.find('年') != -1) & (v.find('ヶ月') != -1) :
            year, month = v.split('年')
            month = month[:-2]
            term.append(float(year) + float(month)/12)
        elif (v.find('年') != -1):
            term.append(float(v[:-1]))
        elif (v.find('ヶ月') != -1):
            term.append(float(v[:-2])/12)
        else:
            term.append(0)
    df[col] = term
    return df

In [114]:
def pre_area(df):
    area=[]
    for i in df['専有面積']:
        area.append(i[:i.find('㎡')])
    df['専有面積']=pd.DataFrame(area)
    return df

In [115]:
def pre_dep_key(df):
    col = '敷金・礼金'
    dep = []
    key = []
    df[col] = df[col].str.replace('円','')
    df[col] = df[col].str.replace(',','')
    for i,v in enumerate(df[col]):
        dep_i, key_i = v.split('・')
        if (dep_i.find('敷金なし') != -1) | (dep_i.find('-') != -1) :
            dep.append(0)
        elif (dep_i.find('ヶ月') != -1):
            d_p = df.loc[i,'price']
            d_m = float(dep_i[:dep_i.find('ヶ月')])
            dep.append(d_p * d_m)
        elif dep_i.find('万') != -1:
            man, sen = dep_i.split('万')
            if sen == '':
                sen = 0
            dep.append(int(man) + int(sen) / 10000)
        else:
            dep.append(int(dep_i))

        if (key_i.find('礼金なし') != -1) | (key_i.find('-') != -1) :
            key.append(0)
        elif (key_i.find('ヶ月') != -1):
            k_p = df.loc[i,'price']
            k_m = float(key_i[:key_i.find('ヶ月')])
            key.append(k_p * k_m)
        elif key_i.find('万') != -1:
            man, sen = key_i.split('万')
            if sen == '':
                sen = 0
            key.append(int(man) + int(sen) / 10000)
        else:
            key.append(int(key_i))

    df['敷金'] = dep
    df['礼金'] = key
    df = df.drop(col, axis = 1)
    return df

In [116]:
def pre_renewal(df):
    col = '更新料'
    df[col] = df[col].str.replace('円','')
    df[col] = df[col].str.replace(',','')
    renewal = []
    for i,v in enumerate(df[col]):
        if (v.find('ヶ月') != -1):
            p = df.loc[i,'price']
            m = float(v[:v.find('ヶ月')])
            renewal.append(p * m)
        elif (v.find('-') != -1):
            renewal.append(0)
        elif v.find('億') != -1:
            oku,man = v.split('億')
            if man == '':
                man = 0
            else:
                man = man[:-1]
            renewal.append(int(oku)*100000000+int(man))
        elif v.find('万') != -1:
            man, sen = v.split('万')
            if sen == '':
                sen = 0
            renewal.append(int(man) + int(sen) / 10000)
        else:
            renewal.append(int(v))
    df[col] = renewal
    return df

In [117]:
def pre_structure(df):
    col = '構造'
    new = []
    for i,v in enumerate(df[col]):
        if len(v.split())==0:
            new.append('-')
        else:
            new.append(v)
    df[col] = new
    df[col]=df[col].str.replace('-','その他')
    df[col]=df[col].str.replace('ブロック造','その他')
    df[col]=df[col].str.replace('プレキャストコンクリート（ＰＣ）','その他')
    df[col]=df[col].str.replace('鉄骨プレキャストコンクリート（ＨＰＣ）','その他')
    df[col]=df[col].str.replace('木造','その他')
    df[col]=df[col].str.replace('軽量鉄骨','その他')
    df[col]=df[col].str.replace('ＡＬＣ','鉄骨造')
    return df

In [118]:
def pre_state(df):
    col = '現況'
    df[col]=df[col].str.replace('-','不明')
    return df

In [135]:
def pre_mcost(df):
    col = '管理費等'
    df[col]=df[col].str.replace(',','')
    df[col]=df[col].str.replace('円','')

    kanri = []
    kyoeki = []
    for i,v in enumerate(df[col]):
        if (v.find('管理費:') != -1) & (v.find('共益費:') != -1):
            kan, kyo = v.split('共益費:')
            if kan[4:] == 'なし':
                kanri.append(0)
            else:
                if kan[4:].find('万')!= -1:
                    man, sen=kan[4:].split('万')
                    if sen=='':
                        sen=0
                    kanri.append(int(man)*10000 + int(sen))
                else:
                    kanri.append(int(kan[4:]))

            if kyo == 'なし':
                kyoeki.append(0)
            else:
                if kyo.find('万')!= -1:
                    man, sen=kyo.split('万')
                    if sen=='':
                        sen=0
                    kyoeki.append(int(man)*10000 + int(sen))
                else:
                    kyoeki.append(int(kyo))

        elif (v.find('管理費:') != -1) & (v.find('共益費:') == -1):
            kyoeki.append(0)
            if v[4:] == 'なし':
                kanri.append(0)
            else:
                if v[4:].find('万')!=-1:
                    man, sen=v[4:].split('万')
                    if sen=='':
                        sen=0
                    kanri.append(int(man)*10000 + int(sen))
                else:
                    kanri.append(int(v[4:]))

        elif (v.find('管理費:') == -1) & (v.find('共益費:') != -1):
            kanri.append(0)
            if v[4:] == 'なし':
                kyoeki.append(0)
            else:
                if v[4:].find('万')!=-1:
                    man, sen=v[4:].split('万')
                    if sen=='':
                        sen=0
                    kyoeki.append(int(man)*10000 + int(sen))
                else:
                    kyoeki.append(int(v[4:]))

        elif (v.find('管理費:') == -1) & (v.find('共益費:') == -1):
            kanri.append(0)
            kyoeki.append(0)

    df['管理費'] = kanri
    df['共益費'] = kyoeki
    df[col] = df['管理費'] + df['共益費']
    df = df.drop(['管理費','共益費'],axis = 1)
    return df

In [120]:
def pre_age(df):
    this_year = datetime.now().strftime("%Y")
    df=df.reset_index(drop=True)
    year=[]
    for i in df['築年月']:
        year.append(int(this_year)-int(i[:i.find('年')]))
    df['築年月']=year
    df=df.rename(columns={'築年月':'築年数'})
    return df

In [127]:
def pre_floor(df):
    df['階数']=df['階数'].str.replace('地下','-')
    df['階数']=df['階数'].str.replace('地上','')
    floor=[]
    for i in df['階数']:
        floor.append(i[:i.find('階')])
    df['階数']=floor
    return df

In [132]:
def encode_cat_to_label(spec):
    mode = 'learning'
#     if self.mode=='inference':
    if mode == 'inference':
        with open(self.encoder_path,'rb') as f:
            labels=pickle.load(f)
#     elif self.mode=='learning':
    elif mode == 'learning':
        labels=get_encoder(spec)
#         self.save_encoder(labels)
    for key in labels.keys():
        spec[key]=labels[key].transform(spec[key])
    return spec

def get_encoder(spec):
    le_company = preprocessing.LabelEncoder()
    le_company.fit(spec['不動産会社詳細'])
    le_insurance = preprocessing.LabelEncoder()
    le_insurance.fit(spec['保険'])
    le_timing = preprocessing.LabelEncoder()
    le_timing.fit(spec['入居時期'])
    le_type = preprocessing.LabelEncoder()
    le_type.fit(spec['取引態様'])
    le_direction = preprocessing.LabelEncoder()
    le_direction.fit(spec['向き'])
    le_structure = preprocessing.LabelEncoder()
    le_structure.fit(spec['構造'])
    le_cat = preprocessing.LabelEncoder()
    le_cat.fit(spec['物件種目'])
    le_state = preprocessing.LabelEncoder()
    le_state.fit(spec['現況'])
    le_layout = preprocessing.LabelEncoder()
    le_layout.fit(spec['間取り'])
    le_way = preprocessing.LabelEncoder()
    le_way.fit(spec['駅からの手段'])
    le_station = preprocessing.LabelEncoder()
    le_station.fit(spec['最寄り駅'])
    labels={
        '不動産会社詳細': le_company,
        '保険': le_insurance,
        '入居時期': le_timing,
        '取引態様': le_type,
        '向き': le_direction,
        '構造':le_structure,
        '物件種目':le_cat,
        '現況':le_state,
        '間取り':le_layout,
        '駅からの手段':le_way,
        '最寄り駅':le_station
       }
    return labels

In [136]:
spec1 = pre_price(spec)
spec2 = spec1.drop(drop_col,axis = 1)
spec3 = pre_bid(spec2)
spec4 = pre_balcony(spec3)
spec5 = pre_transportation(spec4)
spec6 = pre_deposit(spec5)
spec7 = pre_insurance(spec6)
spec8 = pre_timing(spec7)
spec9 = pre_term(spec8)
spec10 = pre_area(spec9)
spec11 = pre_dep_key(spec10)
spec12 = pre_renewal(spec11)
spec13 = pre_structure(spec12)
spec14 = pre_state(spec13)
spec15 = pre_mcost(spec14)
spec16 = pre_age(spec15)
spec17 = pre_floor(spec16)
spec18 = encode_cat_to_label(spec17)

In [141]:
data_version = '20180808'
with open('intermediate_data/preprocessed_rent_for_learning_{}.pickle'.format(data_version),'wb') as f:
        pickle.dump(spec18, f)

In [153]:
read_data =pd.read_csv('sub_data/13TOKYO.CSV', encoding = 'shift-jis' ,header = None)

In [154]:
zip_code = pd.DataFrame(columns = ['郵便番号', '住所'])

In [171]:
zip_code['郵便番号'] = read_data[2]
zip_code['住所'] = read_data[6] + read_data[7] + read_data[8]

In [ ]:
zip_code['住所']

In [193]:
for i in zip_code['住所']:
    if i.find('（') != -1:
        a, b = i.split('（')
        print(b)

次のビルを除く）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）
３４階）
３５階）
３６階）
３７階）
次のビルを除く）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）
３４階）
３５階）
３６階）
次のビルを除く）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）
３４階）
３５階）
３６階）
３７階）
３８階）
３９階）
４０階）
４１階）
４２階）
４３階）
４４階）
１丁目）
２丁目）
次のビルを除く）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）
３４階）
３５階）
３６階）
３７階）
３８階）
３９階）
４０階）
４１階）
４２階）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）

８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）
３４階）
３５階）
３６階）
３７階）
３８階）
３９階）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）
３４階）
３５階）
３６階）
３７階）
３８階）
３９階）
４０階）
４１階）
４２階）
４３階）
４４階）
４５階）
４６階）
４７階）
４８階）
４９階）
５０階）
５１階）
５２階）
５３階）
５４階）
２丁目１番１〜２３号、２番）
その他）
１丁目）
２〜５丁目）
次のビルを除く）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）
３４階）
３５階）
３６階）
３７階）
次のビルを除く）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
１〜４丁目）
５、６丁目）
次のビルを除く）
地階・階層不明）
１階）
２階）
３階）
４階）
５階）
６階）
７階）
８階）
９階）
１０階）
１１階）
１２階）
１３階）
１４階）
１５階）
１６階）
１７階）
１８階）
１９階）
２０階）
２１階）
２２階）
２３階）
２４階）
２５階）
２６階）
２７階）
２８階）
２９階）
３０階）
３１階）
３２階）
３３階）

In [185]:
for i in spec18['所在地']:
    if i.find('丁目') != -1:
        print(i[i.find('丁目')-1:i.find('丁目')])

６
５
１
１
２
１
２
２
９
３
２
３
１
２
５
５
３
３
２
２
２
２
２
１
２
１
３
２
２
１
２
２
３
３
２
５
２
２
４
１
５
８
１
２
３
５
１
２
２
４
１
４
５
４
２
１
１
２
５
４
１
２
１
３
３
６
５
６
１
３
２
３
１
１
２
２
４
５
１
４
３
２
２
３
２
１
３
１
３
６
１
２
４
２
１
３
１
５
１
２
１
２
６
１
５
１
５
１
３
３
１
１
３
１
４
１
２
３
２
７
１
２
３
３
１
２
４
１
１
４
２
１
３
４
２
３
６
２
３
４
１
２
３
５
１
２
７
５
１
２
４
６
２
２
１
５
６
４
２
５
２
３
６
１
５
１
４
４
７
１
５
９
２
１
３
２
３
５
４
１
３
１
６
２
４
５
４
２
７
４
４
３
７
２
１
４
３
２
４
３
４
６
３
２
３
２
２
７
１
２
２
６
１
１
４
２
１
２
３
２
５
１
４
２
３
１
４
２
１
２
１
２
６
３
３
３
４
１
４
３
５
２
７
３
４
２
４
２
４
１
２
３
７
１
３
５
４
６
１
１
３
３
６
１
４
４
２
１
４
２
６
３
２
２
３
２
２
２
３
４
５
１
４
４
４
４
２
３
３
４
３
３
１
１
６
１
１
３
１
２
５
２
６
８
２
２
３
２
１
２
４
２
１
２
２
１
３
２
２
２
２
２
１
２
２
１
２
１
１
１
４
２
２
５
３
６
１
２
２
１
５
２
１
２
４
４
１
２
２
２
１
７
２
１
１
１
３
３
５
５
２
６
５
２
１
３
１
４
３
４
３
２
１
２
４
６
４
３
２
２
７
８
２
５
３
２
５
３
５
４
３
２
１
３
４
１
１
６
５
２
６
１
６
３
３
５
１
１
２
２
１
２
２
２
２
４
２
４
２
４
７
１
２
１
６
１
１
６
５
４
５
４
５
３
３
４
３
２
１
１
１
４
４
２
２
２
４
４
５
１
１
１
２
１
１
２
４
３
２
１
３
１
３
４
２
６
７
７
３
３
２
３
２
４
２
５
１
２
２
３
５
３
３
２
１
１
１
３
３
４
１
６
４
５
３
１
１
１
２
３


４
２
３
２
４
３
２
１
１
４
１
５
２
７
６
６
２
２
２
２
３
１
１
１
１
２
４
５
２
４
２
５
５
１
１
２
１
５
３
３
４
１
１
１
８
３
５
１
１
７
４
３
３
３
３
６
３
１
２
２
２
５
５
５
２
６
６
１
２
１
２
５
１
２
４
３
７
４
４
３
３
２
５
３
２
３
３
１
３
２
１
１
１
４
１
１
２
３
２
２
３
３
４
２
３
５
３
３
３
５
１
１
８
１
１
２
６
２
３
２
５
８
５
４
５
２
２
７
２
２
５
２
４
４
４
２
４
３
２
１
８
４
２
１
１
２
３
１
２
４
３
７
６
３
３
２
４
１
１
１
１
４
４
１
４
２
１
２
５
４
３
６
１
２
２
１
２
３
１
４
２
４
４
４
４
７
２
２
２
２
３
２
４
２
５
１
２
６
３
５
２
５
３
３
４
３
２
５
２
１
２
２
４
３
６
２
４
４
３
１
３
２
２
２
５
１
２
４
７
４
１
２
５
７
４
２
２
２
３
２
１
５
２
３
１
１
６
２
５
２
１
２
１
３
３
５
１
４
１
３
２
５
３
４
２
２
３
１
４
５
５
３
１
４
４
２
３
４
３
３
１
１
１
４
５
１
２
２
４
３
７
２
３
１
２
１
５
３
３
１
３
４
２
４
４
１
５
２
３
１
１
１
５
４
３
３
３
１
５
２
１
２
１
２
３
３
３
４
４
１
３
３
１
３
３
２
５
４
４
３
５
５
２
１
４
１
４
４
４
３
１
５
５
１
２
１
１
２
１
１
６
１
２
３
６
２
５
３
７
４
３
１
３
２
３
２
１
３
７
２
３
５
３
１
２
２
２
３
５
２
２
３
４
２
８
２
６
５
４
６
４
２
１
４
２
１
４
２
３
１
３
３
１
３
４
１
４
５
５
２
２
２
２
１
１
１
１
５
４
６
３
２
１
１
４
１
２
５
２
４
２
５
４
３
４
２
２
４
３
４
４
２
２
３
１
５
３
５
３
４
１
４
７
２
２
３
２
１
２
１
８
１
３
１
２
２
２
１
２
３
２
７
５
４
１
４
２
２
３
６
２
１
３
２
１
１
１
３
４
４


１
４
６
４
８
２
７
２
４
３
４
２
３
１
３
３
４
４
１
５
１
２
１
３
２
３
７
２
１
３
２
４
３
１
２
１
３
１
６
１
３
３
１
３
２
２
２
３
２
４
１
１
３
３
１
３
２
１
５
３
５
３
３
２
７
７
３
５
４
２
１
５
１
１
２
２
１
２
３
２
２
４
２
３
３
３
５
２
３
１
１
２
２
１
１
６
７
９
２
３
２
４
８
３
２
３
３
６
４
３
１
４
２
４
３
１
４
２
３
２
２
５
１
１
１
３
４
６
１
５
１
５
５
４
３
２
８
１
３
２
１
２
５
５
２
２
３
３
５
１
１
２
５
９
３
２
１
１
２
３
３
４
２
８
３
２
２
２
２
２
２
２
４
４
５
２
１
２
１
４
１
３
１
８
３
５
２
２
１
１
７
２
１
２
２
１
１
２
３
４
５
４
２
２
１
５
４
３
２
４
１
３
３
２
３
４
４
３
１
２
６
５
３
３
３
４
１
２
３
６
１
７
１
２
５
２
１
４
１
１
３
６
６
２
３
４
１
２
２
４
３
１
５
１
２
３
４
３
４
７
１
２
７
１
１
３
２
２
１
７
４
４
６
３
２
３
２
２
３
４
６
４
４
５
１
６
１
４
３
１
４
２
２
２
３
５
２
１
２
２
３
１
４
５
３
２
３
１
３
４
２
７
４
２
１
４
３
５
１
５
２
６
３
５
１
３
４
３
１
１
４
１
２
２
１
６
３
２
１
３
３
１
１
３
５
４
１
４
２
３
１
１
２
５
６
２
３
４
２
５
１
２
６
３
４
４
２
５
１
１
３
２
３
２
２
２
２
５
７
２
２
４
３
２
３
６
２
１
２
３
３
３
１
５
２
１
１
２
４
５
５
４
２
２
２
３
２
１
２
２
２
２
４
１
３
１
７
３
２
１
３
１
５
６
１
３
８
１
１
３
２
１
３
６
１
２
１
２
６
３
２
１
３
２
３
５
１
２
３
６
３
２
１
２
３
４
４
１
６
４
２
２
１
５
２
２
１
１
３
５
１
４
５
２
５
１
４
４
２
１
２
３
４
２
２
２
１
８
２
３
４
３
２
２
２
６
２
６
１
３


２
７
２
２
３
２
２
３
５
４
４
１
３
５
２
２
７
４
２
１
２
４
７
４
１
２
１
３
１
１
２
２
３
２
４
３
６
１
６
５
８
２
３
４
１
２
１
６
３
１
３
２
４
２
１
３
４
４
４
７
３
４
２
１
３
５
１
３
２
１
２
１
４
１
３
１
１
２
１
５
１
３
３
４
２
４
４
２
２
１
１
１
１
４
１
１
１
１
１
４
４
３
６
４
７
３
４
１
２
７
７
７
１
３
１
４
４
２
３
４
４
４
２
２
２
１
１
３
３
６
２
１
３
１
４
２
１
１
５
１
５
１
１
３
２
２
４
３
１
１
３
７
２
１
５
１
２
２
１
１
１
２
３
１
２
５
２
３
２
２
４
１
１
３
４
１
６
２
１
４
６
２
６
３
５
６
４
５
６
１
６
５
２
３
１
４
４
５
５
１
２
６
４
１
３
５
３
３
４
４
５
５
２
２
１
２
１
２
３
５
１
２
２
１
１
１
１
４
６
３
２
５
５
５
２
２
４
３
３
１
４
１
４
２
２
２
１
７
４
４
４
３
４
２
２
７
１
４
３
３
２
１
５
２
１
２
１
１
６
６
２
１
３
２
２
１
３
３
７
４
６
２
１
３
２
２
２
２
３
３
１
２
３
２
４
２
２
２
２
２
６
２
３
５
２
３
３
４
３
１
３
４
１
５
２
４
３
３
３
２
３
７
４
６
２
４
５
１
２
２
５
２
２
１
１
２
１
１
２
４
３
２
２
２
６
３
３
３
４
４
１
１
７
４
２
５
２
２
７
８
１
５
２
３
２
２
３
４
３
７
１
７
１
３
５
４
３
１
２
１
２
７
１
１
５
１
１
１
３
３
６
３
２
３
３
１
１
２
１
３
４
３
１
８
２
１
４
２
３
２
４
５
５
１
７
３
４
１
３
１
５
１
２
６
１
５
７
１
２
２
４
２
４
１
３
１
３
４
３
４
２
３
２
１
１
２
２
２
１
４
４
２
３
１
３
１
２
２
１
２
２
７
１
１
２
３
１
１
２
３
３
６
２
１
４
２
３
３
３
１
６
２
１
２
３
１
２
４
２
３
６
２
２
２
５
６
３
４
３
５


２
１
２
２
３
１
１
４
１
４
３
３
４
１
３
１
５
２
１
２
５
１
１
１
２
４
３
２
２
１
１
１
４
４
３
３
１
２
１
５
４
３
３
２
２
５
５
４
３
４
３
４
３
９
２
２
２
４
４
５
３
２
２
１
３
３
３
３
７
１
２
３
２
８
４
１
１
２
６
２
１
３
１
３
３
１
３
７
６
２
５
３
４
２
４
３
２
２
１
２
５
３
６
２
２
５
５
３
５
２
６
７
７
４
４
３
３
４
２
１
１
２
２
３
２
３
４
１
５
５
３
１
２
２
１
３
３
１
７
３
２
１
２
３
５
１
５
２
３
１
３
１
１
３
３
１
２
２
７
２
１
６
３
２
４
３
９
２
２
３
２
４
２
４
２
２
４
２
２
５
３
６
１
１
２
２
５
３
３
１
４
３
２
３
２
６
３
５
１
１
２
３
３
６
６
１
３
１
４
４
６
２
４
４
１
４
３
３
３
５
２
３
２
２
５
１
３
７
１
２
１
３
５
５
２
４
２
６
１
５
３
５
３
１
１
３
５
２
１
２
１
１
３
３
４
４
４
１
２
７
５
１
１
１
２
１
４
２
１
２
３
４
１
２
２
４
２
２
２
３
３
１
７
２
３
２
４
３
４
４
５
１
１
３
２
２
６
３
１
８
２
３
４
１
１
３
１
１
５
３
１
５
１
２
１
３
２
１
５
３
３
２
６
４
１
１
２
５
２
１
４
２
４
３
２
２
１
１
１
４
３
２
４
６
４
１
４
３
４
２
４
１
１
２
４
２
５
３
３
２
６
１
６
１
６
１
１
２
６
２
１
２
５
６
３
１
５
１
１
２
３
１
３
３
１
７
２
１
３
１
６
１
２
２
３
１
５
１
２
３
２
３
２
４
２
４
２
１
１
６
６
４
１
１
６
１
４
３
４
３
３
５
３
１
３
３
４
４
５
２
１
１
１
３
３
３
１
２
３
７
２
２
２
８
１
５
２
１
１
２
４
３
３
２
１
６
５
５
１
１
１
５
２
３
５
５
３
３
４
３
５
３
１
４
３
６
２
２
２
３
１
３
２
４
３
１
３
１
４
１
１
８
４
５
５
２
６
４
４
４


...

７
２
１
５
２
１
３
１
２
２
１
２
５
１
１
３
２
１
１
３
７
２
１
３
１
６
４
３
５
５
３
４
３
２
３
２
５
１
４
３
３
１
１
１
１
４
３
５
４
１
５
３
２
１
３
２
７
１
４
３
１
１
２
３
２
４
１
１
２
６
４
２
３
３
１
４
３
５
２
４
１
３
２
１
４
５
２
３
４
４
１
２
１
２
６
３
３
４
７
７
２
１
１
３
１
１
３
３
５
２
２
１
１
３
２
５
４
２
６
１
３
３
６
１
２
６
１
３
１
８
５
１
１
５
３
８
６
２
１
２
４
１
２
１
３
４
１
１
４
３
２
２
２
２
１
２
３
６
３
１
３
１
２
４
５
３
５
３
２
２
４
４
３
３
３
１
２
２
２
５
１
５
４
３
２
３
１
３
１
１
２
１
３
８
１
１
１
２
６
１
２
４
３
１
３
５
７
１
３
１
３
５
１
１
１
４
１
２
２
１
３
２
２
３
２
２
１
１
４
１
２
２
３
２
４
３
１
３
３
１
３
２
２
２
２
４
３
５
２
４
２
３
３
２
２
５
１
４
２
６
２
３
１
３
４
２
１
４
４
５
１
１
２
３
２
１
３
３
１
４
２
２
２
７
３
３
６
４
４
３
２
１
５
６
３
１
１
１
２
３
２
１
３
２
３
４
３
１
２
２
３
３
２
２
１
２
１
１
５
１
１
３
２
２
３
２
３
１
３
２
３
４
１
２
３
３
１
２
２
５
５
５
２
３
３
２
６
２
２
２
３
６
２
４
２
８
６
３
３
３
６
１
４
２
４
２
２
３
２
４
２
４
３
２
２
３
６
３
４
４
１
３
４
３
５
１
１
３
２
１
３
２
１
１
３
５
２
５
２
１
１
３
１
１
３
１
３
２
２
５
２
２
３
１
５
１
５
２
４
３
２
２
２
２
４
２
５
３
４
３
２
５
１
１
４
１
１
４
１
７
１
１
２
２
２
４
３
５
８
３
４
１
７
１
３
３
６
１
３
２
４
４
３
３
３
６
２
２
２
３
３
６
２
３
１
４
２
７
５
５
２
３
６
６
２
１
８
４
４
２
１
２
６
４
４
３
６
２
１
５


５
２
２
１
１
２
１
３
２
１
６
１
２
４
４
１
２
１
４
４
１
５
３
５
２
２
５
２
２
３
３
２
６
８
２
４
２
４
１
２
３
４
４
１
４
３
２
２
４
３
３
３
２
３
２
３
３
１
３
６
６
１
３
５
２
２
１
３
３
６
１
１
１
８
２
１
１
１
１
４
２
２
２
２
１
１
２
３
７
５
１
４
３
３
１
６
２
２
４
１
１
２
２
１
１
３
４
３
３
２
４
５
２
３
１
３
６
１
１
３
５
１
３
１
３
６
４
２
４
２
１
２
１
２
２
３
４
２
１
１
３
２
３
１
１
４
１
４
３
１
３
５
４
２
３
２
２
４
６
４
１
２
２
４
１
２
２
２
１
２
３
１
４
６
１
１
２
２
１
２
２
３
４
２
４
６
２
４
２
７
３
６
２
３
３
１
３
３
５
２
１
１
３
２
１
１
４
５
２
１
３
６
３
２
１
６
２
２
４
３
３
２
４
１
５
３
２
１
２
１
５
１
３
１
６
１
３
３
２
１
１
１
２
１
１
５
２
３
３
４
６
２
１
２
６
４
３
１
４
２
３
４
２
３
６
５
１
１
４
４
２
３
１
３
４
６
３
３
４
３
２
３
５
２
３
１
４
１
６
２
２
１
３
３
２
２
３
５
２
５
３
２
６
２
１
１
１
３
３
３
３
２
４
１
２
３
１
１
３
３
３
２
１
４
１
４
２
７
３
１
１
１
３
４
３
３
３
２
２
３
２
２
３
３
１
３
１
２
３
３
６
１
４
２
１
１
４
６
１
２
８
１
３
２
８
４
５
１
１
２
２
２
６
３
１
４
３
３
４
５
１
３
４
２
１
１
７
３
２
２
３
４
１
３
７
５
２
２
５
５
３
６
５
８
３
２
１
１
４
４
２
３
２
２
２
２
２
１
２
１
２
６
４
２
２
２
３
１
１
１
３
１
４
１
６
１
３
５
５
３
４
１
１
２
２
２
２
４
４
３
１
３
１
３
３
８
１
３
４
９
５
４
１
２
１
４
２
２
２
６
１
２
３
５
３
４
３
３
５
６
３
１
３
１
１
５
１
２
２
１
２
５
２
２
１
３
１
１
４
２


２
１
２
２
１
２
６
１
３
１
１
２
３
３
１
６
１
４
１
３
２
４
４
３
４
２
２
１
５
２
５
３
１
１
５
３
１
４
４
５
７
３
４
４
３
４
２
３
５
１
１
１
４
２
３
３
１
２
１
４
２
５
５
４
６
７
４
５
３
３
３
５
２
１
２
２
１
１
１
３
４
２
２
３
４
２
２
１
２
５
４
２
３
１
１
１
１
６
３
２
１
５
３
３
１
４
１
２
３
３
１
１
４
１
５
７
６
２
７
１
２
４
２
１
２
１
１
２
２
１
６
３
３
４
６
４
１
４
３
１
３
３
２
１
３
３
２
２
２
５
７
１
６
６
４
２
３
５
３
３
２
４
２
６
５
２
２
３
４
１
２
１
６
４
４
５
３
１
５
３
１
２
３
２
８
１
４
４
１
１
２
３
６
３
３
２
１
４
１
１
２
１
２
２
２
４
７
３
３
２
１
１
３
２
２
２
３
７
１
３
３
４
２
１
１
２
２
１
５
１
４
３
６
１
２
３
１
４
４
４
１
６
８
１
７
４
１
３
３
５
１
４
４
３
３
６
３
２
７
２
３
１
３
２
３
２
４
６
７
２
３
２
１
２
３
４
２
１
１
１
５
４
４
６
４
１
４
１
５
１
４
３
３
２
５
１
３
２
３
１
３
１
１
５
３
１
５
３
３
１
４
１
２
２
７
４
３
１
２
１
５
４
４
４
３
３
１
２
２
５
１
３
４
２
４
２
２
２
４
３
３
２
１
３
１
４
３
５
３
３
１
２
４
３
６
１
５
１
４
２
１
１
１
７
４
３
４
１
２
１
３
３
５
２
１
２
３
１
１
１
２
６
１
６
５
４
２
３
１
２
３
３
２
１
３
１
６
５
３
１
１
１
３
３
５
３
４
１
２
８
３
２
１
８
３
１
３
２
３
２
１
５
９
３
３
３
１
１
４
２
４
１
４
６
５
３
４
４
８
２
５
６
２
２
４
１
１
１
１
１
４
２
１
２
４
４
３
２
４
４
１
３
２
１
１
３
３
３
３
１
３
１
２
３
４
７
３
６
５
５
２
２
１
２
５
４
３
４
３
５
３
３
５
２
１
４
２
１
５
６


２
２
１
６
４
１
４
６
２
５
３
４
１
３
２
２
４
１
２
１
４
３
８
１
２
３
１
２
５
７
２
４
４
４
１
４
１
３
３
１
３
１
３
７
２
１
３
２
１
１
３
４
５
４
２
２
６
１
３
５
５
２
２
４
１
４
９
２
１
５
３
８
３
３
１
２
４
１
４
３
１
２
１
３
２
３
２
２
１
２
２
５
１
１
５
３
１
４
３
３
２
２
３
３
４
２
３
３
１
２
２
３
３
１
２
４
４
２
２
２
４
２
２
７
３
５
２
２
３
２
１
１
４
１
１
１
３
３
２
２
１
２
３
４
３
２
４
３
１
７
３
１
４
３
３
３
２
１
５
４
１
４
３
５
４
４
１
３
１
２
４
２
３
１
３
４
３
１
３
２
４
１
１
４
３
６
４
１
４
２
３
１
４
６
２
２
１
１
８
６
３
６
４
３
１
２
２
２
１
５
４
５
２
６
１
１
３
２
６
６
３
３
２
１
５
５
５
２
１
３
４
２
２
３
５
３
４
４
２
２
１
７
４
４
１
４
５
１
２
２
３
１
２
４
３
３
２
７
１
２
５
２
２
２
４
４
２
１
１
３
５
３
２
２
３
４
４
２
３
３
３
２
５
３
５
２
５
１
４
５
４
１
２
１
４
４
２
５
２
６
３
５
２
３
１
２
５
７
２
３
１
５
１
２
１
２
２
１
４
４
１
２
３
１
３
７
３
２
１
１
３
１
１
４
２
３
２
６
５
５
５
４
３
２
２
３
２
２
５
２
３
９
４
４
１
３
１
１
３
１
３
１
１
３
２
６
４
２
１
２
１
２
２
３
２
３
４
１
６
８
３
４
４
３
２
２
３
３
５
５
３
５
６
３
１
６
４
３
１
１
２
１
２
２
３
５
３
１
２
３
１
４
２
６
７
６
１
１
３
２
６
１
３
６
１
３
１
５
２
５
３
１
１
３
２
３
４
４
３
３
２
３
４
２
５
２
２
３
３
１
１
１
３
７
４
１
３
３
５
２
３
３
１
２
２
１
２
３
５
１
１
３
２
１
１
５
２
４
１
３
４
１
３
１
１
２
２
２
４
３
１
１
１
１
１
２
６
２
１
７


２
１
６
４
４
２
１
７
６
２
２
２
２
３
２
３
６
２
２
４
４
６
１
２
２
６
２
１
１
１
５
４
１
１
２
４
６
２
２
４
４
３
１
１
３
７
５
３
７
５
２
４
３
５
３
２
６
５
１
１
３
２
２
３
１
２
３
５
４
２
２
２
３
２
７
４
１
１
６
５
１
３
１
１
４
３
４
４
６
１
４
３
３
４
２
１
４
５
２
２
３
１
１
１
４
１
３
３
６
３
２
１
２
２
２
７
１
３
３
１
１
１
２
２
３
６
２
４
１
５
４
４
１
２
２
１
１
４
４
３
１
１
７
４
１
１
１
１
１
１
１
３
２
３
２
２
７
３
３
２
２
４
１
２
５
１
６
２
２
６
４
４
２
３
２
２
３
５
２
２
３
１
３
１
３
２
６
２
１
３
２
３
２
３
２
２
７
１
２
５
２
１
４
２
１
２
４
１
３
５
４
６
１
２
１
８
４
３
１
３
１
１
３
５
４
２
２
２
２
２
３
３
３
３
１
４
１
２
２
１
５
３
５
６
２
１
１
７
５
２
４
１
４
３
２
１
１
１
１
２
１
３
１
４
２
１
３
１
２
１
３
２
６
３
５
４
１
６
２
５
２
４
３
２
２
４
３
１
４
２
２
４
７
１
５
４
３
２
１
２
５
３
４
２
１
２
５
１
４
１
３
３
５
４
４
１
１
３
２
２
４
６
１
４
２
５
４
１
１
２
３
１
３
３
４
４
１
１
２
３
３
３
３
３
６
２
５
２
１
１
３
３
１
２
２
３
１
１
２
１
４
２
３
１
３
３
３
１
４
３
４
３
８
４
８
１
１
２
３
３
５
３
５
４
３
５
１
２
２
１
３
６
５
１
１
６
２
１
１
５
７
２
７
６
７
４
３
２
２
２
４
１
２
２
３
１
７
７
２
１
３
３
２
６
５
５
６
５
２
３
２
１
６
２
３
３
８
２
１
３
１
２
１
１
２
４
５
２
５
５
１
４
２
２
４
２
２
２
２
３
４
２
３
１
１
２
１
２
２
２
１
２
４
２
７
６
１
６
１
１
３
３
７
２
４
１
３
４
２
３
２
２
４
７
５
１
１
５
２
１


In [194]:
zip_code

,郵便番号,住所
0,1000000,東京都千代田区以下に掲載がない場合
1,1020072,東京都千代田区飯田橋
2,1020082,東京都千代田区一番町
3,1010032,東京都千代田区岩本町
4,1010047,東京都千代田区内神田
5,1000011,東京都千代田区内幸町
6,1000004,東京都千代田区大手町（次のビルを除く）
7,1006890,東京都千代田区大手町ＪＡビル（地階・階層不明）
8,1006801,東京都千代田区大手町ＪＡビル（１階）
9,1006802,東京都千代田区大手町ＪＡビル（２階）
